In [1]:
import pandas as pd

In [2]:
def read_and_give_dict(path):
    """
        The function returns a Dictionary {"Version", "Defination", "Sequence"}
        for the protiens in a file, given in the path.
        """
    with open(path) as file:
        info = file.read()
    info = info.split(">")
    while True:
        try:
            info.remove("")
        except:
            break        
    protien_list, protien = [], {"Version": [], "Defination": [], "Sequence": []}
    for i in info:
        temp, seq = i.split("\n", maxsplit=1)
        protien['Version'].append(temp.split(" ", maxsplit=1)[0])
        protien['Defination'].append(temp.split(" ", maxsplit=1)[1])
        protien["Sequence"].append("".join(seq.split("\n")))
    return protien

In [3]:
protien = read_and_give_dict("./../Protein_alignment.txt")
len(protien['Sequence'][0])

112

In [4]:
"""
    Import the scoring scheme "BLOSUM62.txt", to score the 
    simliarities between the various protiens to be used in
    making the Phylogenic Tree.
    """
with open("./../BLOSUM62.txt") as file:
    info = file.read()
info = info.split("\n")
info = [ x.split() for x in info ]
scores = pd.DataFrame(info)
temp = list(scores.iloc[0])
temp[0] = "Remove"
scores.columns, scores.index = temp, temp
scores = scores.drop(scores.index[0])
scores = scores.drop(scores.columns[0], axis=1)
scores.head()

,A,R,N,D,C,Q,E,G,H,I,...,P,S,T,W,Y,V,B,Z,X,*
A,4,-1,-2,-2,0,-1,-1,0,-2,-1,...,-1,1,0,-3,-2,0,-2,-1,0,-4
R,-1,5,0,-2,-3,1,0,-2,0,-3,...,-2,-1,-1,-3,-2,-3,-1,0,-1,-4
N,-2,0,6,1,-3,0,0,0,1,-3,...,-2,1,0,-4,-2,-3,3,0,-1,-4
D,-2,-2,1,6,-3,0,2,-1,-1,-3,...,-1,0,-1,-4,-3,-3,4,1,-1,-4
C,0,-3,-3,-3,9,-3,-4,-3,-3,-1,...,-3,-1,-1,-2,-2,-1,-3,-3,-2,-4


In [5]:
def get_score(str1, str2):
    """
        Used to find the degree of difference between 2 given strings
        based on the data from "./../BLOSUM62.txt".
        """
    global scores
    m = len(str2)
    str1 = "*".join( str1.split("-") )
    str2 = "*".join( str2.split("-") )
    score = 0
    for i in range(m):
        score += int(scores.at[str1[i], str2[i]])
    return score

In [6]:
"""
    Building the distance matrix row by row and appending to 
    distance_matrix, to get a final 2d distance array.
    """
distance_matrix = []
for i in range(len(protien['Sequence'])):
    distance = []
    for j in range(len(protien['Sequence'])):
        distance.append( get_score(protien['Sequence'][i], protien['Sequence'][j]) )
    distance_matrix.append(distance)

In [7]:
df = pd.DataFrame(distance_matrix)
names = {0:"A", 1:"B", 2:"C", 3:"D", 4:"E",
         5: "F", 6:"G", 7:"H", 8:"I", 9:"J"}
df.rename(names, inplace=True)
df.columns = list(names.values())
df

,A,B,C,D,E,F,G,H,I,J
A,589,168,255,221,201,200,194,212,210,214
B,168,591,237,332,314,270,332,347,359,356
C,255,237,579,331,346,333,302,331,330,326
D,221,332,331,596,459,425,444,468,485,482
E,201,314,346,459,583,499,423,480,487,484
F,200,270,333,425,499,575,388,451,442,438
G,194,332,302,444,423,388,575,455,481,478
H,212,347,331,468,480,451,455,587,522,518
I,210,359,330,485,487,442,481,522,591,584
J,214,356,326,482,484,438,478,518,584,591


In [8]:
df.to_csv("Pdistance.txt")